In [3]:
import sqlite3
from datetime import datetime, timedelta

In [4]:
conn = sqlite3.connect('library.db')
cur = conn.cursor()

# Function to add a new book
def add_book(title, author, isbn, publication_year, category, availability):
    cur.execute('''
        INSERT INTO Books (Title, Author, ISBN, Publication_Year, Category, Availability)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (title, author, isbn, publication_year, category, availability))
    conn.commit()

# Function to update book information
def update_book(book_id, title=None, author=None, isbn=None, publication_year=None, category=None, availability=None):
    command = 'UPDATE Books SET '
    parameters = []
    if title is not None:
        command += 'Title = ?, '
        parameters.append(title)
    if author is not None:
        command += 'Author = ?, '
        parameters.append(author)
    if isbn is not None:
        command += 'ISBN = ?, '
        parameters.append(isbn)
    if publication_year is not None:
        command += 'Publication_Year = ?, '
        parameters.append(publication_year)
    if category is not None:
        command += 'Category = ?, '
        parameters.append(category)
    if availability is not None:
        command += 'Availability = ? '
        parameters.append(availability)
    command += 'WHERE Book_ID = ?'
    parameters.append(book_id)

    cur.execute(command, parameters)
    conn.commit()

# Function to search for books
def search_books(search_term):
    search_term = f'%{search_term}%'
    cur.execute('''
        SELECT * FROM Books WHERE
        Title LIKE ? OR Author LIKE ? OR Category LIKE ?
    ''', (search_term, search_term, search_term))
    return cur.fetchall()

# Function to mark a book as borrowed
def borrow_book(book_id, student_faculty_id, librarian_id):
    cur.execute('''
        INSERT INTO Transactions (Book_ID, Student_Faculty_ID, Librarian_ID, Borrowed_Date)
        VALUES (?, ?, ?, ?)
    ''', (book_id, student_faculty_id, librarian_id, datetime.now()))
    cur.execute('''
        UPDATE Books SET Availability = 'No' WHERE Book_ID = ?
    ''', (book_id,))
    conn.commit()

# Function to return a book
def return_book(transaction_id):
    cur.execute('''
        UPDATE Transactions SET Returned_Date = ? WHERE Transaction_ID = ?
    ''', (datetime.now(), transaction_id))
    # Get the book ID from the transaction to update the book availability
    cur.execute('''
        SELECT Book_ID FROM Transactions WHERE Transaction_ID = ?
    ''', (transaction_id,))
    book_id = cur.fetchone()[0]
    cur.execute('''
        UPDATE Books SET Availability = 'Yes' WHERE Book_ID = ?
    ''', (book_id,))
    conn.commit()

# Function to view borrowing history
def view_borrowing_history(student_faculty_id):
    cur.execute('''
        SELECT * FROM Transactions WHERE Student_Faculty_ID = ?
    ''', (student_faculty_id,))
    return cur.fetchall()

# Function to generate report on book availability
def report_book_availability():
    cur.execute('SELECT * FROM Books WHERE Availability = "Yes"')
    return cur.fetchall()

# Function to generate report on overdue books
def report_overdue_books():
    today = datetime.now()
    cur.execute('''
        SELECT * FROM Transactions WHERE Returned_Date IS NULL AND Borrowed_Date < ?
    ''', (today - timedelta(days=30),))  # Assuming 30 days as the threshold for overdue
    return cur.fetchall()

# Function to generate report on borrowing trends (Example: Most borrowed books)
def report_borrowing_trends():
    cur.execute('''
        SELECT Transactions.Book_ID, Transactions.Title, COUNT(Transactions.Book_ID) AS Count
        FROM Transactions
        INNER JOIN Books ON Books.BookID = Transactions.Book_ID
        GROUP BY Transactions.Book_ID
        ORDER BY Count DESC
        LIMIT 10
    ''')
    return cur.fetchall()

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()
